Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [3]:
# подгружаем .env
load_dotenv()

True

In [4]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [5]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [6]:
# Таблица Здания
TABLE = 'buildings'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, dst_conn)
data

,id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,6220,1965,6,55.717113,37.781120,2.64,84,12,True
1,18012,2001,2,55.794849,37.608013,3.00,97,10,True
2,17821,2000,4,55.740040,37.761742,2.70,80,10,True
3,18579,2002,4,55.672016,37.570877,2.64,771,17,True
4,9293,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...
24615,22074,2012,2,55.737213,37.493084,3.00,132,22,True
24616,14084,1984,4,55.626541,37.720547,2.64,361,16,True
24617,13923,1983,4,55.798008,37.801689,2.64,270,17,True
24618,18289,2001,4,55.884998,37.535870,2.65,118,17,True


In [114]:
# Таблица квартиры
TABLE = 'flats'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, dst_conn)
data

,id,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id
0,0,9,False,9.90,19.900000,1,False,35.099998,9500000,6220
1,1,7,False,0.00,16.600000,1,False,43.000000,13500000,18012
2,2,9,False,9.00,32.000000,2,False,56.000000,13500000,17821
3,3,1,False,10.10,43.099998,3,False,76.000000,20000000,18579
4,4,3,False,3.00,14.000000,1,False,24.000000,5200000,9293
...,...,...,...,...,...,...,...,...,...,...
141357,141357,16,False,11.00,18.000000,1,False,42.000000,10500000,22455
141358,141358,5,False,5.28,28.330000,2,False,41.110001,7400000,3162
141359,141359,7,False,5.30,20.000000,1,False,31.500000,9700000,6513
141360,141360,15,False,13.80,33.700001,2,False,65.300003,11750000,23952


# Колонки, которые влияют
Buildings:

build_year
building_type_int
latitude
longitude
ceiling_height
flats_count
floors_total
has_elevator

building_id*

flats:
floor
kitchen_area
living_area
rooms
is_apartment
studio
total_area
price - TARGET

In [159]:
sql = f"""
select f.id as flat_id, f.price, f.floor, f.kitchen_area, f.living_area, f.rooms, f.is_apartment, f.studio, f.total_area,
    b.build_year, b.building_type_int, b.latitude, b.longitude, b.ceiling_height, b.flats_count,
    b.floors_total, b.has_elevator
from buildings as b left join flats as f on
    f.building_id = b.id
"""
result_table = pd.read_sql(sql, dst_conn)
result_table

,float_id,price,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9500000,9,9.90,19.900000,1,False,False,35.099998,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,13500000,7,0.00,16.600000,1,False,False,43.000000,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,13500000,9,9.00,32.000000,2,False,False,56.000000,2000,4,55.740040,37.761742,2.70,80,10,True
3,3,20000000,1,10.10,43.099998,3,False,False,76.000000,2002,4,55.672016,37.570877,2.64,771,17,True
4,4,5200000,3,3.00,14.000000,1,False,False,24.000000,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141357,141357,10500000,16,11.00,18.000000,1,False,False,42.000000,2013,4,55.626579,37.313503,2.64,672,25,True
141358,141358,7400000,5,5.28,28.330000,2,False,False,41.110001,1960,1,55.727470,37.768677,2.48,80,5,False
141359,141359,9700000,7,5.30,20.000000,1,False,False,31.500000,1966,4,55.704315,37.506584,2.64,72,9,True
141360,141360,11750000,15,13.80,33.700001,2,False,False,65.300003,2017,4,55.699863,37.939564,2.70,480,25,True


# Transform

In [160]:
# Удаление дубликатов
feature_cols = result_table.columns.drop('float_id').tolist()
is_duplicated_features = result_table.duplicated(subset=feature_cols, keep=False)
result_table = result_table[~is_duplicated_features].reset_index(drop=True)
result_table

,float_id,price,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9500000,9,9.90,19.900000,1,False,False,35.099998,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,13500000,7,0.00,16.600000,1,False,False,43.000000,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,13500000,9,9.00,32.000000,2,False,False,56.000000,2000,4,55.740040,37.761742,2.70,80,10,True
3,3,20000000,1,10.10,43.099998,3,False,False,76.000000,2002,4,55.672016,37.570877,2.64,771,17,True
4,4,5200000,3,3.00,14.000000,1,False,False,24.000000,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123932,141357,10500000,16,11.00,18.000000,1,False,False,42.000000,2013,4,55.626579,37.313503,2.64,672,25,True
123933,141358,7400000,5,5.28,28.330000,2,False,False,41.110001,1960,1,55.727470,37.768677,2.48,80,5,False
123934,141359,9700000,7,5.30,20.000000,1,False,False,31.500000,1966,4,55.704315,37.506584,2.64,72,9,True
123935,141360,11750000,15,13.80,33.700001,2,False,False,65.300003,2017,4,55.699863,37.939564,2.70,480,25,True


In [161]:
result_table.min()

float_id                     0
price                       11
floor                        1
kitchen_area               0.0
living_area                0.0
rooms                        1
is_apartment             False
studio                   False
total_area                11.0
build_year                1901
building_type_int            0
latitude              55.21146
longitude            36.864372
ceiling_height             2.0
flats_count                  1
floors_total                 1
has_elevator             False
dtype: object

# Удаление пустых значений

In [162]:
import numpy as np
cols_to_replace = ['kitchen_area', 'living_area', 'total_area']
num_cols =result_table[cols_to_replace]
num_cols = num_cols.replace(0, np.nan)
result_table[cols_to_replace] = num_cols
result_table

,float_id,price,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9500000,9,9.90,19.900000,1,False,False,35.099998,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,13500000,7,NaN,16.600000,1,False,False,43.000000,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,13500000,9,9.00,32.000000,2,False,False,56.000000,2000,4,55.740040,37.761742,2.70,80,10,True
3,3,20000000,1,10.10,43.099998,3,False,False,76.000000,2002,4,55.672016,37.570877,2.64,771,17,True
4,4,5200000,3,3.00,14.000000,1,False,False,24.000000,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123932,141357,10500000,16,11.00,18.000000,1,False,False,42.000000,2013,4,55.626579,37.313503,2.64,672,25,True
123933,141358,7400000,5,5.28,28.330000,2,False,False,41.110001,1960,1,55.727470,37.768677,2.48,80,5,False
123934,141359,9700000,7,5.30,20.000000,1,False,False,31.500000,1966,4,55.704315,37.506584,2.64,72,9,True
123935,141360,11750000,15,13.80,33.700001,2,False,False,65.300003,2017,4,55.699863,37.939564,2.70,480,25,True


In [163]:
cols_with_nans = result_table.isnull().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0].index
cols_with_nans

Index(['kitchen_area', 'living_area'], dtype='object')

In [164]:
for col in cols_with_nans:
    if result_table[col].dtype in [float, int]:
        fill_value = result_table[col].median()
    elif result_table[col].dtype == 'object':
        fill_value = result_table[col].mode().iloc[0]
    result_table[col] = result_table[col].fillna(fill_value)

In [165]:
result_table

,float_id,price,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9500000,9,9.90,19.900000,1,False,False,35.099998,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,13500000,7,9.00,16.600000,1,False,False,43.000000,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,13500000,9,9.00,32.000000,2,False,False,56.000000,2000,4,55.740040,37.761742,2.70,80,10,True
3,3,20000000,1,10.10,43.099998,3,False,False,76.000000,2002,4,55.672016,37.570877,2.64,771,17,True
4,4,5200000,3,3.00,14.000000,1,False,False,24.000000,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123932,141357,10500000,16,11.00,18.000000,1,False,False,42.000000,2013,4,55.626579,37.313503,2.64,672,25,True
123933,141358,7400000,5,5.28,28.330000,2,False,False,41.110001,1960,1,55.727470,37.768677,2.48,80,5,False
123934,141359,9700000,7,5.30,20.000000,1,False,False,31.500000,1966,4,55.704315,37.506584,2.64,72,9,True
123935,141360,11750000,15,13.80,33.700001,2,False,False,65.300003,2017,4,55.699863,37.939564,2.70,480,25,True


# Удаление выбросов

In [166]:
result_table.max()

float_id                 141361
price                9873737728
floor                        56
kitchen_area              203.0
living_area               700.0
rooms                        20
is_apartment               True
studio                    False
total_area                920.0
build_year                 2023
building_type_int             6
latitude              56.011032
longitude             37.946411
ceiling_height             27.0
flats_count                4455
floors_total                 99
has_elevator               True
dtype: object

In [167]:
# Удаление выбросов
num_cols = result_table.select_dtypes(['float', 'int']).columns
num_cols = num_cols.drop(['latitude', 'longitude', 'building_type_int', 'build_year', 'float_id'])
num_cols

Index(['price', 'floor', 'kitchen_area', 'living_area', 'rooms', 'total_area',
       'ceiling_height', 'flats_count', 'floors_total'],
      dtype='object')

In [168]:
threshold = 1.5
potential_outliers = pd.DataFrame()

for col in num_cols:
    Q1 = result_table[col].quantile(0.25)
    Q3 = result_table[col].quantile(0.75)
    IQR = Q3 - Q1
    margin = threshold * IQR
    lower = Q1 - margin
    upper = Q3 + margin
    potential_outliers[col] = ~result_table[col].between(lower, upper)


In [169]:
outliers = potential_outliers.any(axis=1)
result_table = result_table[~outliers].reset_index(drop=True)
result_table

,float_id,price,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9500000,9,9.90,19.900000,1,False,False,35.099998,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,13500000,7,9.00,16.600000,1,False,False,43.000000,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,13500000,9,9.00,32.000000,2,False,False,56.000000,2000,4,55.740040,37.761742,2.70,80,10,True
3,4,5200000,3,3.00,14.000000,1,False,False,24.000000,1971,1,55.808807,37.707306,2.60,208,9,True
4,5,8490104,9,9.00,30.799999,2,False,False,51.009998,2017,4,55.724728,37.743069,2.70,192,17,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97141,141356,10800000,8,6.00,42.000000,3,False,False,64.000000,1971,4,55.740402,37.834579,2.64,428,9,True
97142,141358,7400000,5,5.28,28.330000,2,False,False,41.110001,1960,1,55.727470,37.768677,2.48,80,5,False
97143,141359,9700000,7,5.30,20.000000,1,False,False,31.500000,1966,4,55.704315,37.506584,2.64,72,9,True
97144,141360,11750000,15,13.80,33.700001,2,False,False,65.300003,2017,4,55.699863,37.939564,2.70,480,25,True


In [170]:
result_table.max()

float_id                 141361
price                  28400000
floor                        20
kitchen_area              14.97
living_area           70.099998
rooms                         5
is_apartment               True
studio                    False
total_area           118.199997
build_year                 2023
building_type_int             6
latitude              56.011032
longitude             37.946411
ceiling_height              3.0
flats_count                 638
floors_total                 29
has_elevator               True
dtype: object

In [171]:
result_table.min()

float_id                     0
price                       11
floor                        1
kitchen_area              2.25
living_area                2.0
rooms                        1
is_apartment             False
studio                   False
total_area                11.0
build_year                1902
building_type_int            0
latitude              55.21146
longitude            36.864372
ceiling_height            2.45
flats_count                  1
floors_total                 1
has_elevator             False
dtype: object

In [188]:
# Удаление выбросов
num_cols = result_table[['price']].columns
num_cols

Index(['price'], dtype='object')

In [189]:
threshold = 1.5
potential_outliers = pd.DataFrame()

for col in num_cols:
    Q1 = result_table[col].quantile(0.25)
    Q3 = result_table[col].quantile(0.75)
    IQR = Q3 - Q1
    margin = threshold * IQR
    lower = Q1 - margin
    upper = Q3 + margin
    potential_outliers[col] = ~result_table[col].between(lower, upper)

In [190]:
outliers = potential_outliers.any(axis=1)
result_table = result_table[~outliers].reset_index(drop=True)
result_table

,float_id,price,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9500000,9,9.90,19.900000,1,False,False,35.099998,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,13500000,7,9.00,16.600000,1,False,False,43.000000,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,13500000,9,9.00,32.000000,2,False,False,56.000000,2000,4,55.740040,37.761742,2.70,80,10,True
3,4,5200000,3,3.00,14.000000,1,False,False,24.000000,1971,1,55.808807,37.707306,2.60,208,9,True
4,5,8490104,9,9.00,30.799999,2,False,False,51.009998,2017,4,55.724728,37.743069,2.70,192,17,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93751,141356,10800000,8,6.00,42.000000,3,False,False,64.000000,1971,4,55.740402,37.834579,2.64,428,9,True
93752,141358,7400000,5,5.28,28.330000,2,False,False,41.110001,1960,1,55.727470,37.768677,2.48,80,5,False
93753,141359,9700000,7,5.30,20.000000,1,False,False,31.500000,1966,4,55.704315,37.506584,2.64,72,9,True
93754,141360,11750000,15,13.80,33.700001,2,False,False,65.300003,2017,4,55.699863,37.939564,2.70,480,25,True


In [172]:
result_table.sort_values(by=['price']) 

,float_id,price,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
48468,69633,11,9,6.0,33.000000,2,False,False,42.0,2007,4,55.724037,37.907486,2.78,368,24,True
74271,107198,19,2,9.0,21.000000,2,False,False,70.0,1972,1,55.796005,37.734650,2.64,288,9,True
72479,104573,85,9,8.0,46.000000,1,False,False,59.0,1964,1,55.813976,37.713226,2.64,108,9,True
42658,60417,1500,3,9.0,30.799999,1,False,False,36.0,2012,4,55.677567,37.666252,2.80,180,10,True
53743,76800,2000,4,9.0,20.000000,1,False,False,38.0,1966,1,55.844582,37.491531,2.48,198,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38487,54315,28400000,2,9.0,45.000000,3,False,False,60.0,1928,1,55.752060,37.580742,3.00,54,6,True
44133,62551,28400000,4,10.3,61.599998,4,False,False,96.0,2006,4,55.712551,37.754387,2.64,584,17,True
61679,88756,28400000,1,10.0,25.000000,2,False,False,53.5,2016,0,55.799774,37.710899,3.00,482,11,False
82213,120220,28400000,7,10.0,63.000000,4,False,False,99.0,1987,4,55.631935,37.520992,2.64,336,22,True


In [191]:
result_table[result_table['price'] < 1000000]

,float_id,price,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
37260,54185,891450,2,9.0,30.799999,1,False,False,36.0,1979,4,55.856655,37.432716,2.65,127,16,True
41938,61167,600000,13,9.0,30.299999,2,False,False,52.0,1979,4,55.872162,37.627140,2.64,128,16,True
53090,78484,700000,3,8.0,18.000000,3,False,False,56.0,1963,1,55.835312,37.520561,2.48,119,5,False


In [174]:
#Удалить пустые значения - 0 в столбцах floor, kitchen_area